In [ ]:
import sklearn
from keras.layers import LSTM, GRU, Dense, Dropout, Bidirectional
from keras.models import Sequential
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.metrics import cohen_kappa_score
import numpy as np
import pickle


In [ ]:
with open('org_glv_w2v.pkl', 'rb') as file:
  glv_vec = pickle.load(file)
  w2v_vec = pickle.load(file)
  y = pickle.load(file)
  random_state = pickle.load(file)
  test_size = pickle.load(file)
file.close()

In [ ]:
with open('org_gen_aug_glv.pkl', 'rb') as file:
  new_x_train_glv = pickle.load(file)
  new_y_train_glv = pickle.load(file)
  aug_vec_glv = pickle.load(file)
  aug_y_glv = pickle.load(file)
file.close()

In [ ]:
from sklearn.model_selection import RandomizedSearchCV


In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

In [ ]:
X_train_glv, X_test_glv, y_train_glv, y_test_glv = train_test_split(glv_vec, y, test_size=0.25, random_state=41)

In [ ]:
def rnn(RNN, input_shape):
  if RNN == 'LSTM': RNN = LSTM
  elif RNN == 'GRU': RNN = GRU
  
  """Define the model."""
  model = Sequential()
  model.add(Bidirectional(RNN(input_shape[-1], dropout=0.4, recurrent_dropout=0.4, input_shape=[1, input_shape[-1]], return_sequences=True)))
  model.add(Bidirectional(RNN(64, recurrent_dropout=0.4)))
  model.add(Dropout(0.5))
  model.add(Dense(1, activation='relu'))

  model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
  model.build(input_shape)
  model.summary()

  return model

In [ ]:
rnn1 = rnn('GRU', X_train_glv.shape)
rnn1.fit(X_train_glv, y_train_glv, batch_size=32, epochs=100)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_4 (Bidirectio  (8037, 1, 100)           30600     
 nal)                                                            
                                                                 
 bidirectional_5 (Bidirectio  (8037, 128)              63744     
 nal)                                                            
                                                                 
 dropout_2 (Dropout)         (8037, 128)               0         
                                                                 
 dense_2 (Dense)             (8037, 1)                 129       
                                                                 
Total params: 94,473
Trainable params: 94,473
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
252/252 [=============================

In [ ]:
rnn1 = rnn('LSTM', X_train_glv.shape)
rnn1.fit(X_train_glv, y_train_glv, batch_size=64, epochs=100)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (8037, 1, 100)           40400     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (8037, 128)              84480     
 nal)                                                            
                                                                 
 dropout (Dropout)           (8037, 128)               0         
                                                                 
 dense (Dense)               (8037, 1)                 129       
                                                                 
Total params: 125,009
Trainable params: 125,009
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
126/126 [=============================

In [ ]:
def test_rnn_model(model, x_test, y_test):
  y_pred = model.predict(x_test)
  y_pred = np.around(y_pred)

  result_aug = cohen_kappa_score(y_test,y_pred,weights='quadratic')
  print("Kappa Score: {}".format(result_aug))

In [ ]:
#LSTM
test_rnn_model(rnn1, X_test_glv, y_test_glv)

84/84 [==============================] - 2s 5ms/step
Kappa Score: 0.6966068923949904


In [ ]:
#GRU
test_rnn_model(rnn1, X_test_glv, y_test_glv)

84/84 [==============================] - 2s 4ms/step
Kappa Score: 0.7012565233891114


In [ ]:
_, new_x_test_glv, _, new_y_test_glv = train_test_split(new_x_train_glv, new_y_train_glv, test_size=0.25)

In [ ]:
#LSTM
test_rnn_model(rnn1, new_x_test_glv, new_y_test_glv)

67/67 [==============================] - 0s 3ms/step
Kappa Score: 0.0718160004233811


In [ ]:
#GRU
test_rnn_model(rnn1, new_x_test_glv, new_y_test_glv)

65/65 [==============================] - 0s 3ms/step
Kappa Score: 0.6920043769200634


In [ ]:
aug_x_tr, aug_x_ts, aug_y_tr, aug_y_ts = train_test_split(aug_vec_glv, aug_y_glv, test_size=0.2)

In [ ]:
def rnn(RNN, input_shape):
  if RNN == 'LSTM': RNN = LSTM
  elif RNN == 'GRU': RNN = GRU
  
  """Define the model."""
  model = Sequential()
  model.add(Bidirectional(RNN(input_shape[-1], dropout=0.2, recurrent_dropout=0.2, input_shape=[1, input_shape[-1]], return_sequences=True)))
  model.add(Bidirectional(RNN(64, recurrent_dropout=0.2)))
  model.add(Dropout(0.2))
  model.add(Dense(1, activation='relu'))

  model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
  model.build(input_shape)
  model.summary()

  return model

In [ ]:
rnn = rnn('LSTM', aug_x_tr.shape)
rnn.fit(aug_x_tr, y_train_glv, batch_size=64, epochs=100)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_2 (Bidirectio  (13245, 1, 100)          40400     
 nal)                                                            
                                                                 
 bidirectional_3 (Bidirectio  (13245, 128)             84480     
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (13245, 128)              0         
                                                                 
 dense_1 (Dense)             (13245, 1)                129       
                                                                 
Total params: 125,009
Trainable params: 125,009
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
207/207 [===========================

In [ ]:
rnn_gru = rnn('GRU', aug_x_tr.shape)
rnn_gru.fit(aug_x_tr, aug_y_tr, batch_size=64, epochs=100)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_10 (Bidirecti  (13072, 1, 100)          30600     
 onal)                                                           
                                                                 
 bidirectional_11 (Bidirecti  (13072, 128)             63744     
 onal)                                                           
                                                                 
 dropout_5 (Dropout)         (13072, 128)              0         
                                                                 
 dense_5 (Dense)             (13072, 1)                129       
                                                                 
Total params: 94,473
Trainable params: 94,473
Non-trainable params: 0
_________________________________________________________________
Epoch 1/80
205/205 [==============================

In [ ]:
#LSTM
test_rnn_model(rnn, aug_x_ts, aug_y_ts)

104/104 [==============================] - 2s 4ms/step
Kappa Score: 0.7665505303754433


In [ ]:
#GRU
test_rnn_model(rnn_gru, aug_x_ts, aug_y_ts)

103/103 [==============================] - 1s 3ms/step
Kappa Score: 0.7092647093361628


In [ ]:
#LSTM
test_rnn_model(rnn, X_test_glv, y_test_glv)

84/84 [==============================] - 1s 6ms/step
Kappa Score: 0.7022976808667822


In [ ]:
#GRU
test_rnn_model(rnn_gru, X_test_glv, y_test_glv)

84/84 [==============================] - 0s 3ms/step
Kappa Score: 0.6735899254398994
